Recently, I learned XGBoost by watching a series of the amazing videos hosted by [StatQuest with Josh Starmer](https://www.youtube.com/watch?v=OtD8wVaFm6E). In this notebook, I summary what I have learned.

# 1. How does XGBoost work

XGBoost builds a model by construcing a series of decision binary trees with each trees fitted to the residual of the previous trees. The output of XGBoost is the combination of these trees. The initial tree, ususally, is just a naive guess. However, after several trees are built, XGBoost can fit the traing data very well.
The residual is just the difference between the targets and predictions of the trees that have been constructed.

Then in order understand how XGBoost works, we should understand two questions:
1. how to split a leaf
1. how to make a prediction.


Since, on the surface, the other two answers are different on regression and classification,
XGBoost will be explained in Regression and Classification respectively.

**Denote the residuals of the $n$ samples in a leaf by $r_i$ with i=1,...,n.**

## 1.1 Split the leaf by using Similarity and Gain
Then we can calculate the **Similarity Score**, $S$ of the leaf.  
For a regression, it is
\begin{equation}
 S = { (\Sigma_{i=1}^n r_i)^2 \over \lambda + n }.
\end{equation}
For a binary classification, it is
$$
 S = { (\Sigma_{i=1}^n r_i)^2 \over \lambda + \Sigma_{j=1}^n p_j(1-p_j)}.
$$
Here $\lambda$ is the regularization parameter. And $p_j$ is the prediction of the $j$th sample in the leaf
given by the trees that have been constructed. Or in other words, $p_j$ is the prediction of the $j$th sample used 
to calculate its residual, $r_j$.

Then we try to find a threshold to split the leaf into two leaves (left and right)
and calculate the similarities scores on the two sub-leaves and denoting them as $S_l$ and $S_r$.
Then we can calculate the **Gain** for this split:
$$
G = S_l + S_r - S.
$$
Then we can try different thresholds to split the leaf and select the threshold with the largest gain
to construct the leaf in the tree. Repeating this procedure, we can construnct a binary tree fitted to the residuals.
But be sure, $G$ must be positive. Otherwise the leaf would not be split.

## 1.2 Calculate the predictin
If this leaf is a terminal node, we can calculate the output of the leaf.
For a regression, it is 
$$
  O = {\Sigma_{i=1}^n r_i \over \lambda + n }.
$$
If $\lambda =0$, this is just the average of the residuals in this leaf.  
For a binary classifiction, it is 
$$
  O = { \Sigma_{i=1}^n r_i \over \lambda + \Sigma_{j=1}^n p_j(1-p_j)}.
$$

Then the prediction of the built trees is 